<a href="https://colab.research.google.com/github/dkanzariya/AI-Notebooks/blob/master/MIMO_NOMA_DATA_GENERATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [187]:
import numpy as np
import math

In [188]:
N = 8
M = 8
SNR_db = np.array(list(range(0 ,11 ,2)))
batch_size = 1

In [189]:
def generate_data(M, batch_size):

    input_ = [[0 for _ in range(pow(2, M)) ]for _ in range(M)]
    label = [[0 for _ in range(pow(2, M)) ]for _ in range( 2 *M)]

    for i in range( 1<<M):
        for j in range(M):
            if i & ( 1<<j):
                input_[ M - j -1][i] = 1

    for i in range( 1<<M):
        for j in range(M):
            if input_[j][i] == 1:
                label[ 2 * j][i] = 1
                label[ 2 * j + 1][i] = 0
            else:
                label[ 2 * j][i] = 0
                label[ 2 * j + 1][i] = 1

    input_ = np.tile(np.array(input_) ,batch_size)
    # label = np.tile(np.eye(1 << M), batch_size) #cnn
    label =  np.tile(np.array(label) ,batch_size)

    return input_, label

In [190]:
def generate(M, N, batch_size):

    data, label = generate_data(M, batch_size)
  
    ran1 = np.random.permutation(batch_size * pow(2, M))  # Suffling Dataset
    ran2 = np.random.permutation(batch_size * pow(2, M))
    
    symbol1 = 2 * data[:, ran1] - 1
    symbol2 = 2 * data[:, ran2] - 1

    SPC = math.sqrt(0.8) * symbol1 + math.sqrt(0.2) * symbol2  # Superposition Coding

    label1 = np.transpose(label[:, ran1].astype('float32'))
    label2 = np.transpose(label[:, ran2].astype('float32'))

    return SPC, label1, label2

In [191]:
def generate_input(H1_real, H1_image, SPC, N, batch_size, sigma):

    N_real, N_image = generate_channel(N, batch_size * pow(2, N), 0)

    input_real = np.matmul(H1_real, SPC) + sigma * N_real
    input_img = np.matmul(H1_image, SPC) + sigma * N_image
    
    input1 = np.transpose(np.concatenate((input_real, input_img), axis=0))

    return input1

In [192]:
def generate_channel(N, M, k):

    h1 = np.random.randn(N, M) / math.sqrt(2)
    h2 = np.random.randn(N, M) / math.sqrt(2)

    if k == 0:
        return h1, h2
    else:
        return 2 * h1, 2 * h2

In [193]:
H1_real, H1_image = generate_channel(N, M, 0)
H2_real, H2_image = generate_channel(N, M, 1)

print(H1_image.shape)
print(H1_real.shape)
print(H2_real.shape)
print(H2_image.shape)

(8, 8)
(8, 8)
(8, 8)
(8, 8)


In [194]:
for i in range(len(SNR_db)):

        SPC, label_user1, label_user2 = generate(M, N, batch_size)
        signal_power = np.mean(pow(SPC, 2))
        sigma_user  = math.sqrt(signal_power / (math.sqrt(M) * pow(10, float(SNR_db[i]) / 10.0)))
        input = generate_input(H2_real, H2_image, SPC, M, batch_size, sigma_user)

print(input.shape , label_user1.shape ,label_user2.shape, SPC.shape, signal_power, sigma_user)

(256, 16) (256, 16) (256, 16) (8, 256) 1.0234375 0.19022087114333788
